<a href="https://colab.research.google.com/github/j2nhyeok/self-study/blob/main/5_3_type2(blank).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 심장마비 확률이 높은사람?
- 성별, 나이, 혈압, 콜레스테롤, 공복혈당, 최대 심박수 등의 컬럼이 있음
- 평가: ROC-AUC, 정확도(Accuracy), F1 을 구하시오
- target : output (1:심장마비 확률 높음, 0:심장마비 확률 낮음)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,output
41,0.633
28,0.123
222,0.355
~~~



In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 데이터 불러오기

In [39]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/빅분기 실기/인프런_data/heart/heart/2files/train.csv')
test = pd.read_csv('/content/drive/MyDrive/빅분기 실기/인프런_data/heart/heart/2files/test.csv')

print(train.shape, test.shape)


(242, 15) (61, 14)


##EDA

In [40]:
train.head(2)

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,48,53,0,2,128,216,0,0,115,0,0.0,2,0,0,1
1,124,39,0,2,94,199,0,1,179,0,0.0,2,0,2,1


In [41]:
test.head(2)

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,41,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,28,65,0,2,140,417,1,0,157,0,0.8,2,1,2


In [42]:
train['output'].value_counts()

1    126
0    116
Name: output, dtype: int64

In [43]:
train.describe()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.00000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,154.706612,54.516529,0.673554,0.913223,131.070248,246.933884,0.119835,0.53719,149.590909,0.347107,1.019835,1.380165,0.785124,2.347107,0.520661
std,86.328576,9.021513,0.469885,1.020897,17.716978,52.236414,0.325441,0.53183,21.763201,0.477037,1.182137,0.621285,1.052218,0.607160,0.500608
min,0.000000,34.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.00000,88.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,81.250000,47.250000,0.000000,0.000000,120.000000,212.000000,0.000000,0.00000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,158.500000,55.000000,1.000000,1.000000,130.000000,243.000000,0.000000,1.00000,152.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,228.750000,61.000000,1.000000,2.000000,140.000000,274.000000,0.000000,1.00000,165.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,302.000000,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.00000,195.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


## 데이터 전처리 & 피처엔지니어링

In [44]:
train = train.drop('id', axis = 1)
test_id = test.pop('id')

test.head(1)


,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,48,1,1,130,245,0,0,180,0,0.2,1,0,2


## 검증 데이터 분리

In [45]:
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(train.drop('output', axis = 1), train['output'], test_size = 0.15, random_state = 2022)
print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)

(205, 13) (37, 13) (205,) (37,)


## 모델 & 평가

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

rf = RandomForestClassifier(random_state = 2022, max_depth=5, n_estimators = 200)
rf = rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
pred_proba = rf.predict_proba(X_val)

print("accuracy : ", accuracy_score(y_val, pred))
print("f1 : ", f1_score(y_val, pred))

# max_depth = 3,5,7,
# n_estimators = 100, 200, 400,800, 1000
# accuracy :  0.7567567567567568
# f1 :        0.7906976744186046
# accuracy :  0.8378378378378378   max_depth = 3
# f1 :  0.8636363636363636
# accuracy :  0.8108108108108109   max_depth = 5
# f1 :  0.8444444444444444
# accuracy :  0.8108108108108109   max_depth = 7
# f1 :  0.8444444444444444

print("roc_auc : ", roc_auc_score(y_val, pred_proba[:,1]))
# rf  0.9301242236024845
# roc_auc :  0.9378881987577639     max_depth = 3
# roc_auc :  0.9409937888198757     max_depth = 5 /  roc_auc :  0.9440993788819876 !200 /  0.9409937888198757 !400
# roc_auc :  0.9347826086956521     max_depth = 7

accuracy :  0.8108108108108109
f1 :  0.8444444444444444
roc_auc :  0.9440993788819876


In [47]:
from xgboost import XGBClassifier
xgb = XGBClassifier(room_state = 2022, max_depth=3, n_estimators=600, learning_rate=0.01)
xgb.fit(X_tr, y_tr)
pred = xgb.predict(X_val)
pred_proba = xgb.predict_proba(X_val)

print("accuracy : ", accuracy_score(y_val, pred))
print("f1 : ", f1_score(y_val, pred))
print("roc_auc : ", roc_auc_score(y_val, pred_proba[:,1]))

# max_depth 3,4,5,6,7,8,9,12
# n_estimators 100 ~ 1000
# estimator 올리면 learning_rate 낮추기 0.1 ~0.01

# learning_rate
# base
# accuracy :  0.8108108108108109
# f1 :  0.8444444444444444
# roc_auc :  0.9192546583850931

# accuracy :  0.8108108108108109
# f1 :  0.8444444444444444
# roc_auc :  0.906832298136646       200

# accuracy :  0.8108108108108109
# f1 :  0.8444444444444444
# roc_auc :  0.8881987577639752     400/0.05

# accuracy :  0.8108108108108109
# f1 :  0.8444444444444444
# roc_auc :  0.9161490683229814       400/0.01
# accuracy :  0.8108108108108109
# f1 :  0.8444444444444444
# roc_auc :  0.9192546583850931       600/0.01
#________________________________________
# max_depth : 5
# accuracy :  0.7837837837837838
# f1 :  0.8181818181818182
# roc_auc :  0.906832298136646

# max_depth : 7
# accuracy :  0.7837837837837838
# f1 :  0.8181818181818182
# roc_auc :  0.9099378881987576

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [07:41:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "room_state" } are not used.

  warnings.warn(smsg, UserWarning)


accuracy :  0.8108108108108109
f1 :  0.8444444444444444
roc_auc :  0.9192546583850931


## 예측 및 csv 제출

In [48]:
test.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,65,1,3,138,282,1,0,174,0,1.4,1,1,2
3,69,1,3,160,234,1,0,131,0,0.1,1,1,2
4,50,0,1,120,244,0,1,162,0,1.1,2,0,2


In [49]:
pred_proba = xgb.predict_proba(test)

In [50]:
pd.DataFrame({
    'id':test_id,
    'output': pred_proba[:, 1]
}).to_csv('123.csv', index=False)

##정리

In [51]:
# y_test = pd.read_csv("y_test.csv")
# print(roc_auc_score(y_test, pred_proba[:,1]))